In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from pathlib import Path
import logging
import toml
import copy

from simreaduntil.shared_utils.logging_utils import add_comprehensive_stream_handler_to_logger, setup_logger_simple
from simreaduntil.seqsum_tools.seqsum_plotting import preprocess_seqsum_df_for_plotting


add_comprehensive_stream_handler_to_logger(None)
logging.getLogger(__name__).setLevel(logging.DEBUG)
logging.getLogger("simreaduntil").setLevel(logging.DEBUG)

logger = logging

# logging.getLogger(None).setLevel(logging.ERROR)
# logging.getLogger("simreaduntil").setLevel(logging.ERROR)

In [15]:
# run_dir = Path("/home/mmordig/ont_project_all/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_withflanking/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_accel5/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_fakemapper_accel10/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_withunaligned_constantgapsampler_accel5/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_withunaligned_accel10/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_withunaligned_accel5/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_withunaligned_accel2/simulator_run/")
# run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_withunaligned_accel3_longer/simulator_run/")
run_dir = Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_realreads_realmapper_withunaligned_accel5_longer/simulator_run/")

seqsum_filename = run_dir / "sequencing_summary.txt"
sim_config = {"readfish_config_file": run_dir / ".." / "configs/readfish_enrich_per_quadrant.toml" }

readfish_conditions = [v for v in toml.load(sim_config["readfish_config_file"])["conditions"].values() if isinstance(v, dict)]
channel_assignments_toml = run_dir / "channels.toml"
channel_assignments_per_cond = toml.load(channel_assignments_toml)
channels_per_condition = {condition_dict["name"]: condition_dict["channels"] for condition_dict in channel_assignments_per_cond["conditions"].values()}

logger.debug(f"Reading sequencing summary file '{seqsum_filename}'")
full_seqsum_df = pd.read_csv(seqsum_filename, sep="\t")#, nrows=100) # todo
logger.debug(f"Done reading sequencing summary file '{seqsum_filename}'")


In [16]:
# partial_seqsum_df.columns
# full_seqsum_df["channel"].unique()

In [17]:


num_sequenced_bps_per_group_per_condition = {}

# targets of conditions are disjoint, so we can group by each of them for each condition (a condition is a selseq strategy applied to a subset of channels)
group_to_units = {cond["name"]: cond["targets"] for cond in readfish_conditions if cond["name"] != "control"}

for condition in readfish_conditions:
    condition_name = condition["name"]
    print(f"Processing condition {condition_name}")
    subchannels = channels_per_condition[condition_name]
    
    partial_seqsum_df = full_seqsum_df[full_seqsum_df["channel"].isin([f"ch{i}" for i in subchannels])]
    # partial_seqsum_df = full_seqsum_df[full_seqsum_df["channel"].isin([i for i in subchannels])]
    
    partial_seqsum_df, group_column, chrom_column = preprocess_seqsum_df_for_plotting(partial_seqsum_df, group_to_units=copy.deepcopy(group_to_units))
    
    num_sequenced_bps_per_group = dict(partial_seqsum_df.groupby("group")["cum_nb_seq_bps_per_group"].max())
    num_sequenced_bps_per_group_per_condition[condition_name] = num_sequenced_bps_per_group
    

2024-03-01 10:33:16,409 - Sorting and cleaning seqsummary file of shape (184996, 13) --- seqsum_plotting.py:939 (preprocess_seqsum_df_for_plotting) INFO ##


Processing condition control


2024-03-01 10:33:16,677 - Adding previous gap duration to seqsummary --- seqsum_plotting.py:941 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:16,835 - Adding group column from NanoSim read id --- seqsum_plotting.py:951 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:17,431 - Splitting according to groups {'enrich_chr_1_8': ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8'], 'enrich_chr_9_14': ['chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14'], 'enrich_chr_16_20': ['chr16', 'chr17', 'chr18', 'chr19', 'chr20'], 'other': {'chr21', 'chrY', 'chr15', 'chr22', 'chrX'}} --- seqsum_plotting.py:964 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:17,453 - Adding extra columns for plotting --- seqsum_plotting.py:971 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:17,543 - /tmp/ipykernel_1011868/1748468773.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas.

Processing condition enrich_chr_1_8


2024-03-01 10:33:18,095 - Adding previous gap duration to seqsummary --- seqsum_plotting.py:941 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:18,389 - Adding group column from NanoSim read id --- seqsum_plotting.py:951 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:19,486 - Splitting according to groups {'enrich_chr_1_8': ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8'], 'enrich_chr_9_14': ['chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14'], 'enrich_chr_16_20': ['chr16', 'chr17', 'chr18', 'chr19', 'chr20'], 'other': {'chr21', 'chrY', 'chr15', 'chr22', 'chrX'}} --- seqsum_plotting.py:964 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:19,527 - Adding extra columns for plotting --- seqsum_plotting.py:971 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:19,697 - /tmp/ipykernel_1011868/1748468773.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas.

Processing condition enrich_chr_9_14


2024-03-01 10:33:20,519 - Adding previous gap duration to seqsummary --- seqsum_plotting.py:941 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:20,981 - Adding group column from NanoSim read id --- seqsum_plotting.py:951 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:22,599 - Splitting according to groups {'enrich_chr_1_8': ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8'], 'enrich_chr_9_14': ['chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14'], 'enrich_chr_16_20': ['chr16', 'chr17', 'chr18', 'chr19', 'chr20'], 'other': {'chr21', 'chrY', 'chr15', 'chr22', 'chrX'}} --- seqsum_plotting.py:964 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:22,658 - Adding extra columns for plotting --- seqsum_plotting.py:971 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:22,903 - /tmp/ipykernel_1011868/1748468773.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas.

Processing condition enrich_chr_16_20


2024-03-01 10:33:23,805 - Adding previous gap duration to seqsummary --- seqsum_plotting.py:941 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:24,419 - Adding group column from NanoSim read id --- seqsum_plotting.py:951 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:26,596 - Splitting according to groups {'enrich_chr_1_8': ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8'], 'enrich_chr_9_14': ['chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14'], 'enrich_chr_16_20': ['chr16', 'chr17', 'chr18', 'chr19', 'chr20'], 'other': {'chr21', 'chrY', 'chr15', 'chrM', 'chr22', 'chrX'}} --- seqsum_plotting.py:964 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:26,675 - Adding extra columns for plotting --- seqsum_plotting.py:971 (preprocess_seqsum_df_for_plotting) INFO ##
2024-03-01 10:33:27,002 - /tmp/ipykernel_1011868/1748468773.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of

In [18]:
# when having one condition/target per simulation run

# group_to_units = {
#     'enrich_chr_1_8': ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8'],
#     'enrich_chr_9_14': ['chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14'],
#     'enrich_chr_16_20': ['chr16', 'chr17', 'chr18', 'chr19', 'chr20'],
# }
# seqsum_filenames_per_cond = {
#     "control": Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_fakemapper_control/simulator_run/sequencing_summary.txt"),
#     "enrich_chr_1_8": Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_fakemapper_chr1to8/simulator_run/sequencing_summary.txt"),
#     "enrich_chr_9_14": Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_fakemapper_chr9to14/simulator_run/sequencing_summary.txt"),
#     "enrich_chr_16_20": Path("/is/cluster-test/fast/mmordig/ont_project/runs/enrich_usecase/readfish_exp/results_readfishexp_fakemapper_chr16to20/simulator_run/sequencing_summary.txt"),
# }

# assert(all(x.exists() for x in seqsum_filenames_per_cond.values()))

# num_sequenced_bps_per_group_per_condition = {}
# for (condition_name, seqsum_filename) in seqsum_filenames_per_cond.items():
#     partial_seqsum_df, group_column = preprocess_seqsum_df_for_plotting(seqsum_filename, group_to_units=copy.deepcopy(group_to_units))
    
#     num_sequenced_bps_per_group = dict(partial_seqsum_df.groupby("group")["cum_nb_seq_bps_per_group"].max())
#     num_sequenced_bps_per_group_per_condition[condition_name] = num_sequenced_bps_per_group

In [19]:
for (condition_name, num_sequenced_bps_per_group) in num_sequenced_bps_per_group_per_condition.items():
    total_bps = sum(num_sequenced_bps_per_group.values())
    throughput_reduction = sum(num_sequenced_bps_per_group_per_condition["control"].values()) / sum(num_sequenced_bps_per_group.values())
    
    bps_per_target_str = ", ".join(f"{group}: {num_bps:.2E}" for (group, num_bps) in num_sequenced_bps_per_group.items())
    print(f"{condition_name:20}: total: {total_bps:.2E} ({throughput_reduction:.3}), {bps_per_target_str}")

control             : total: 2.67E+09 (1.0), enrich_chr_16_20: 3.35E+08, enrich_chr_1_8: 1.32E+09, enrich_chr_9_14: 6.60E+08, other: 3.54E+08
enrich_chr_1_8      : total: 2.12E+09 (1.26), enrich_chr_16_20: 6.32E+07, enrich_chr_1_8: 1.84E+09, enrich_chr_9_14: 1.33E+08, other: 8.11E+07
enrich_chr_9_14     : total: 1.94E+09 (1.38), enrich_chr_16_20: 9.16E+07, enrich_chr_1_8: 3.60E+08, enrich_chr_9_14: 1.35E+09, other: 1.36E+08
enrich_chr_16_20    : total: 1.75E+09 (1.53), enrich_chr_16_20: 9.05E+08, enrich_chr_1_8: 4.61E+08, enrich_chr_9_14: 2.40E+08, other: 1.43E+08


In [20]:
from simreaduntil.simulator.gap_sampling.gap_sampler_per_window_until_blocked import dict_without_items

absolute_enrichment = {
    condition_name: seq_bps_per_target[condition_name] / num_sequenced_bps_per_group_per_condition["control"][condition_name]
    for (condition_name, seq_bps_per_target) in dict_without_items(num_sequenced_bps_per_group_per_condition, ["control"]).items()
}

# relative composition when no selective sequencing is happening
composition_noselseq = {
    "enrich_chr_1_8": 0.496,
    "enrich_chr_9_14": 0.247,
    "enrich_chr_16_20": 0.125
}
relative_enrichment = {
    condition_name: (seq_bps_per_target[condition_name] / sum(seq_bps_per_target.values())) / composition_noselseq[condition_name]
    for (condition_name, seq_bps_per_target) in dict_without_items(num_sequenced_bps_per_group_per_condition, ["control"]).items()
}

for (condition_name, enrichment) in absolute_enrichment.items():
    print(f"Absolute enrichment of target in condition {condition_name:20}: {enrichment:.3}")
    print(f"Relative enrichment of target in condition {condition_name:20}: {relative_enrichment[condition_name]:.3}")

Absolute enrichment of target in condition enrich_chr_1_8      : 1.4
Relative enrichment of target in condition enrich_chr_1_8      : 1.75
Absolute enrichment of target in condition enrich_chr_9_14     : 2.04
Relative enrichment of target in condition enrich_chr_9_14     : 2.82
Absolute enrichment of target in condition enrich_chr_16_20    : 2.7
Relative enrichment of target in condition enrich_chr_16_20    : 4.14


```
# aligned, fake mapper, accel10 (results_readfishexp_realreads_fakemapper_accel10)
control             : total: 2.72E+09 (1.0 ), enrich_chr_16_20: 3.36E+08, enrich_chr_1_8: 1.35E+09, enrich_chr_9_14: 6.74E+08, other: 3.59E+08
enrich_chr_1_8      : total: 2.28E+09 (1.19), enrich_chr_16_20: 2.09E+07, enrich_chr_1_8: 2.20E+09, enrich_chr_9_14: 4.03E+07, other: 2.19E+07
enrich_chr_9_14     : total: 1.99E+09 (1.36), enrich_chr_16_20: 3.27E+07, enrich_chr_1_8: 1.31E+08, enrich_chr_9_14: 1.79E+09, other: 3.49E+07
enrich_chr_16_20    : total: 1.69E+09 (1.61), enrich_chr_16_20: 1.35E+09, enrich_chr_1_8: 1.92E+08, enrich_chr_9_14: 9.56E+07, other: 5.17E+07
Absolute enrichment of target in condition enrich_chr_1_8      : 1.63
Relative enrichment of target in condition enrich_chr_1_8      : 1.94
Absolute enrichment of target in condition enrich_chr_9_14     : 2.66
Relative enrichment of target in condition enrich_chr_9_14     : 3.65
Absolute enrichment of target in condition enrich_chr_16_20    : 4.0
Relative enrichment of target in condition enrich_chr_16_20    : 6.39

# aligned, realmapper, accel5 (results_readfishexp_realreads_realmapper_accel5)
control             : total: 2.72E+09 (1.0 ), enrich_chr_16_20: 3.38E+08, enrich_chr_1_8: 1.35E+09, enrich_chr_9_14: 6.70E+08, other: 3.65E+08
enrich_chr_1_8      : total: 2.31E+09 (1.18), enrich_chr_16_20: 3.01E+07, enrich_chr_1_8: 2.16E+09, enrich_chr_9_14: 6.96E+07, other: 4.90E+07
enrich_chr_9_14     : total: 2.03E+09 (1.34), enrich_chr_16_20: 4.51E+07, enrich_chr_1_8: 1.71E+08, enrich_chr_9_14: 1.72E+09, other: 1.01E+08
enrich_chr_16_20    : total: 1.73E+09 (1.58), enrich_chr_16_20: 1.28E+09, enrich_chr_1_8: 2.27E+08, enrich_chr_9_14: 1.31E+08, other: 9.46E+07
Absolute enrichment of target in condition enrich_chr_1_8      : 1.59
Relative enrichment of target in condition enrich_chr_1_8      : 1.89
Absolute enrichment of target in condition enrich_chr_9_14     : 2.56
Relative enrichment of target in condition enrich_chr_9_14     : 3.42
Absolute enrichment of target in condition enrich_chr_16_20    : 3.78
Relative enrichment of target in condition enrich_chr_16_20    : 5.9

# with unaligned, realmapper accel10 (results_readfishexp_realreads_realmapper_withunaligned_accel10)
control             : total: 2.67E+09 (1.0), enrich_chr_16_20: 3.32E+08, enrich_chr_1_8: 1.33E+09, enrich_chr_9_14: 6.57E+08, other: 3.55E+08
enrich_chr_1_8      : total: 2.32E+09 (1.15), enrich_chr_16_20: 2.26E+08, enrich_chr_1_8: 1.40E+09, enrich_chr_9_14: 4.52E+08, other: 2.44E+08
enrich_chr_9_14     : total: 2.31E+09 (1.16), enrich_chr_16_20: 2.40E+08, enrich_chr_1_8: 9.53E+08, enrich_chr_9_14: 8.48E+08, other: 2.68E+08
enrich_chr_16_20    : total: 2.27E+09 (1.18), enrich_chr_16_20: 4.90E+08, enrich_chr_1_8: 1.00E+09, enrich_chr_9_14: 5.00E+08, other: 2.74E+08
Absolute enrichment of target in condition enrich_chr_1_8      : 1.06
Relative enrichment of target in condition enrich_chr_1_8      : 1.22
Absolute enrichment of target in condition enrich_chr_9_14     : 1.29
Relative enrichment of target in condition enrich_chr_9_14     : 1.49
Absolute enrichment of target in condition enrich_chr_16_20    : 1.48
Relative enrichment of target in condition enrich_chr_16_20    : 1.73

# with unaligned, realmapper accel2 (results_readfishexp_realreads_realmapper_withunaligned_accel2)
control             : total: 2.67E+09 (1.0), enrich_chr_16_20: 3.34E+08, enrich_chr_1_8: 1.32E+09, enrich_chr_9_14: 6.57E+08, other: 3.52E+08
enrich_chr_1_8      : total: 2.15E+09 (1.24), enrich_chr_16_20: 6.55E+07, enrich_chr_1_8: 1.86E+09, enrich_chr_9_14: 1.37E+08, other: 8.44E+07
enrich_chr_9_14     : total: 1.92E+09 (1.39), enrich_chr_16_20: 9.18E+07, enrich_chr_1_8: 3.61E+08, enrich_chr_9_14: 1.34E+09, other: 1.33E+08
enrich_chr_16_20    : total: 1.74E+09 (1.54), enrich_chr_16_20: 8.91E+08, enrich_chr_1_8: 4.61E+08, enrich_chr_9_14: 2.40E+08, other: 1.44E+08
Absolute enrichment of target in condition enrich_chr_1_8      : 1.41
Relative enrichment of target in condition enrich_chr_1_8      : 1.75
Absolute enrichment of target in condition enrich_chr_9_14     : 2.03
Relative enrichment of target in condition enrich_chr_9_14     : 2.81
Absolute enrichment of target in condition enrich_chr_16_20    : 2.67
Relative enrichment of target in condition enrich_chr_16_20    : 4.11

# with unaligned, realmapper, accel3 (results_readfishexp_realreads_realmapper_withunaligned_accel3_longer)
control             : total: 2.65E+09 (1.0), enrich_chr_16_20: 3.28E+08, enrich_chr_1_8: 1.32E+09, enrich_chr_9_14: 6.47E+08, other: 3.49E+08
enrich_chr_1_8      : total: 2.13E+09 (1.24), enrich_chr_16_20: 6.44E+07, enrich_chr_1_8: 1.85E+09, enrich_chr_9_14: 1.33E+08, other: 8.19E+07
enrich_chr_9_14     : total: 1.92E+09 (1.38), enrich_chr_16_20: 9.07E+07, enrich_chr_1_8: 3.55E+08, enrich_chr_9_14: 1.34E+09, other: 1.33E+08
enrich_chr_16_20    : total: 1.73E+09 (1.53), enrich_chr_16_20: 8.92E+08, enrich_chr_1_8: 4.56E+08, enrich_chr_9_14: 2.39E+08, other: 1.42E+08
Absolute enrichment of target in condition enrich_chr_1_8      : 1.4
Relative enrichment of target in condition enrich_chr_1_8      : 1.75
Absolute enrichment of target in condition enrich_chr_9_14     : 2.07
Relative enrichment of target in condition enrich_chr_9_14     : 2.82
Absolute enrichment of target in condition enrich_chr_16_20    : 2.72
Relative enrichment of target in condition enrich_chr_16_20    : 4.13

# with unaligned, realmapper, accel5 (results_readfishexp_realreads_realmapper_withunaligned_accel5_longer)
control             : total: 2.67E+09 (1.0), enrich_chr_16_20: 3.35E+08, enrich_chr_1_8: 1.32E+09, enrich_chr_9_14: 6.60E+08, other: 3.54E+08
enrich_chr_1_8      : total: 2.12E+09 (1.26), enrich_chr_16_20: 6.32E+07, enrich_chr_1_8: 1.84E+09, enrich_chr_9_14: 1.33E+08, other: 8.11E+07
enrich_chr_9_14     : total: 1.94E+09 (1.38), enrich_chr_16_20: 9.16E+07, enrich_chr_1_8: 3.60E+08, enrich_chr_9_14: 1.35E+09, other: 1.36E+08
enrich_chr_16_20    : total: 1.75E+09 (1.53), enrich_chr_16_20: 9.05E+08, enrich_chr_1_8: 4.61E+08, enrich_chr_9_14: 2.40E+08, other: 1.43E+08
Absolute enrichment of target in condition enrich_chr_1_8      : 1.4
Relative enrichment of target in condition enrich_chr_1_8      : 1.75
Absolute enrichment of target in condition enrich_chr_9_14     : 2.04
Relative enrichment of target in condition enrich_chr_9_14     : 2.82
Absolute enrichment of target in condition enrich_chr_16_20    : 2.7
Relative enrichment of target in condition enrich_chr_16_20    : 4.14


## unused 

# short: with unaligned, realmapper accel5 (results_readfishexp_realreads_realmapper_withunaligned_accel5)
control             : total: 1.82E+09 (1.0), enrich_chr_16_20: 2.23E+08, enrich_chr_1_8: 9.07E+08, enrich_chr_9_14: 4.47E+08, other: 2.39E+08
enrich_chr_1_8      : total: 1.54E+09 (1.18), enrich_chr_16_20: 4.62E+07, enrich_chr_1_8: 1.34E+09, enrich_chr_9_14: 9.68E+07, other: 5.85E+07
enrich_chr_9_14     : total: 1.36E+09 (1.33), enrich_chr_16_20: 6.51E+07, enrich_chr_1_8: 2.55E+08, enrich_chr_9_14: 9.48E+08, other: 9.54E+07
enrich_chr_16_20    : total: 1.27E+09 (1.43), enrich_chr_16_20: 6.53E+08, enrich_chr_1_8: 3.34E+08, enrich_chr_9_14: 1.74E+08, other: 1.05E+08
Absolute enrichment of target in condition enrich_chr_1_8      : 1.47
Relative enrichment of target in condition enrich_chr_1_8      : 1.75
Absolute enrichment of target in condition enrich_chr_9_14     : 2.12
Relative enrichment of target in condition enrich_chr_9_14     : 2.82
Absolute enrichment of target in condition enrich_chr_16_20    : 2.93
Relative enrichment of target in condition enrich_chr_16_20    : 4.13

# for constant gap sampler (results_readfishexp_realreads_realmapper_withunaligned_constantgapsampler_accel5)
control             : total: 2.27E+09 (1.0), enrich_chr_16_20: 2.82E+08, enrich_chr_1_8: 1.12E+09, enrich_chr_9_14: 5.63E+08, other: 3.00E+08
enrich_chr_1_8      : total: 2.21E+09 (1.03), enrich_chr_16_20: 6.70E+07, enrich_chr_1_8: 1.92E+09, enrich_chr_9_14: 1.40E+08, other: 8.49E+07
enrich_chr_9_14     : total: 2.15E+09 (1.05), enrich_chr_16_20: 1.03E+08, enrich_chr_1_8: 4.05E+08, enrich_chr_9_14: 1.49E+09, other: 1.51E+08
enrich_chr_16_20    : total: 2.08E+09 (1.09), enrich_chr_16_20: 1.07E+09, enrich_chr_1_8: 5.56E+08, enrich_chr_9_14: 2.88E+08, other: 1.72E+08
Absolute enrichment of target in condition enrich_chr_1_8      : 1.71
Relative enrichment of target in condition enrich_chr_1_8      : 1.75
Absolute enrichment of target in condition enrich_chr_9_14     : 2.65
Relative enrichment of target in condition enrich_chr_9_14     : 2.81
Absolute enrichment of target in condition enrich_chr_16_20    : 3.78
Relative enrichment of target in condition enrich_chr_16_20    : 4.1
```

In [21]:
# # yield-corrected
# 1.49 * 1.23/1.24,\
# 2.2 * 1.44/1.89, \
# 3.04 * 1.7/2.84
1.41 * 1.24/1.24,\
2.03 * 1.39/1.89, \
2.67 * 1.54/2.84

(1.41, 1.4929629629629628, 1.4478169014084508)

In [ ]:
# yield-corrected
# 1.59 * 1.15/1.24, \
# 2.38 * 1.32/1.89, \
# 3.23 * 1.56/2.84